# AI(2A) - Human Robot Interaction Project
# 👩‍⚕️  Nurse Robot

## Introduction
This script implements a Spoke Dialog System (SDS) based on Spoken Natural Language Dialog in the context of an automated home nurse assistant, to be used by people with limited capabilities. It simulates actions such as reaching things, provide meals and inspect patient status. The system acquires a spoken input from the user through Google Speech Recognition API. Then, recognizes the intent of the sentence using Rasa NLU and retrieves and action result that is transmitted to the user through VocalWare Text-To-Speech API. The order of the sentences, and the recognition of the utterances and dependencies is processed by the libraries spaCy and Keras, building a model that returns the adequate actions given a series of inputs. 

![title](images/NurseRobotMindmap.png)

In [1]:
#imports for project

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import logging
import rasa_core
from rasa_core.agent import Agent
from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.policies.memoization import MemoizationPolicy
from rasa_core.interpreter import RasaNLUInterpreter
from rasa_nlu.model import Metadata, Interpreter
from rasa_core.utils import EndpointConfig
from rasa_core.run import serve_application
from rasa_nlu.model import Trainer
from rasa_nlu.training_data import load_data
from rasa_core import config
from IPython.display import IFrame
import pandas as pd
import ruamel
import urllib.request
from urllib.request import urlopen
from playsound import playsound
import uuid
import speech_recognition as sr
import os
import spacy
from spacy import displacy
import warnings
warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)
warnings.filterwarnings("ignore",category=DeprecationWarning)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' #To ignore Tensorflow AVX AVX2 bonary warning

nlp = spacy.load('en')


Some of the data used in this project is not shown in this notebook, but in external files. The content of each relevant file is described hereunder.

* nurse_domain.yml: Defines the actions that can be taken by the agent, the intents that are expected, the entities to be saved if received, and some answers to the basic user utterances.
* stories.md: Contains the examples of dialogs and the specification of the entities that could be found on the phrases.
* config_spacy.json: Saves the pipeline to be used by the NLP library, in this case sklearn with spaCy 


In [2]:
# Declare paths
domain_file = './nurse_domain.yml'
model_path = './models/dialogue'
interpreter_path ='./models/nursebot/interpreter'
training_data_file = './data/stories.md'
conf_file = './config_spacy.json'

In [3]:
# Build the interpreter of the elements of dialog for the agent
#!python -m rasa_nlu.train -c config_spacy.json --data data/data.md -o models --fixed_model_name interpreter --project nursebot --verbose

In [4]:
action_endpoint = EndpointConfig(url="http://localhost:5055/webhook")
interpreter = RasaNLUInterpreter(interpreter_path)

agent = Agent(domain_file, 
              policies = [MemoizationPolicy(), KerasPolicy(max_history=3, epochs=200, batch_size=50)],
              interpreter=interpreter,
              action_endpoint=action_endpoint)
data = agent.load_data(training_data_file)
agent.train(data)

Processed Story Blocks: 100%|███████████████████████████████████████████| 30/30 [00:01<00:00, 26.92it/s, # trackers=20]
Processed actions: 2616it [00:25, 102.96it/s, # examples=2540]


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 3, 36)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8832      
_________________________________________________________________
dense (Dense)                (None, 16)                528       
_________________________________________________________________
activation (Activation)      (None, 16)                0         
Total params: 9,360
Trainable params: 9,360
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
1437/1437 [==============================] - ETA: 1:28 - loss: 2.7506 - acc: 0.160 - ETA: 15s - loss: 2.7004 - acc: 0.260 - ETA: 8s - loss: 2.6687 - acc: 0.3250 - ETA: 5s - loss: 2.6516 - acc: 0.345 - ETA: 3s - loss: 2.6385 - acc: 0.350 - ETA: 2s - loss: 2.615

1437/1437 [==============================] - ETA: 0s - loss: 1.0544 - acc: 0.740 - ETA: 0s - loss: 1.0147 - acc: 0.752 - ETA: 0s - loss: 0.9806 - acc: 0.751 - ETA: 0s - loss: 0.9817 - acc: 0.746 - ETA: 0s - loss: 0.9530 - acc: 0.754 - ETA: 0s - loss: 0.9450 - acc: 0.755 - ETA: 0s - loss: 0.9461 - acc: 0.752 - ETA: 0s - loss: 0.9451 - acc: 0.750 - ETA: 0s - loss: 0.9384 - acc: 0.751 - ETA: 0s - loss: 0.9258 - acc: 0.757 - 1s 395us/step - loss: 0.9195 - acc: 0.7599
Epoch 16/200
1437/1437 [==============================] - ETA: 0s - loss: 0.9196 - acc: 0.780 - ETA: 0s - loss: 0.8484 - acc: 0.780 - ETA: 0s - loss: 0.8495 - acc: 0.785 - ETA: 0s - loss: 0.8370 - acc: 0.790 - ETA: 0s - loss: 0.8545 - acc: 0.790 - ETA: 0s - loss: 0.8452 - acc: 0.793 - ETA: 0s - loss: 0.8513 - acc: 0.787 - ETA: 0s - loss: 0.8500 - acc: 0.786 - ETA: 0s - loss: 0.8483 - acc: 0.788 - ETA: 0s - loss: 0.8387 - acc: 0.791 - 1s 386us/step - loss: 0.8404 - acc: 0.7905
Epoch 17/200
1437/1437 [===========================

1437/1437 [==============================] - ETA: 0s - loss: 0.2769 - acc: 0.980 - ETA: 0s - loss: 0.2452 - acc: 0.966 - ETA: 0s - loss: 0.2830 - acc: 0.940 - ETA: 0s - loss: 0.2641 - acc: 0.940 - ETA: 0s - loss: 0.2768 - acc: 0.928 - ETA: 0s - loss: 0.2678 - acc: 0.933 - ETA: 0s - loss: 0.2669 - acc: 0.938 - ETA: 0s - loss: 0.2658 - acc: 0.937 - ETA: 0s - loss: 0.2629 - acc: 0.941 - ETA: 0s - loss: 0.2724 - acc: 0.939 - ETA: 0s - loss: 0.2777 - acc: 0.937 - 1s 442us/step - loss: 0.2747 - acc: 0.9388
Epoch 32/200
1437/1437 [==============================] - ETA: 0s - loss: 0.3176 - acc: 0.940 - ETA: 0s - loss: 0.2608 - acc: 0.955 - ETA: 0s - loss: 0.2824 - acc: 0.942 - ETA: 0s - loss: 0.2776 - acc: 0.934 - ETA: 0s - loss: 0.2616 - acc: 0.938 - ETA: 0s - loss: 0.2531 - acc: 0.941 - ETA: 0s - loss: 0.2456 - acc: 0.942 - ETA: 0s - loss: 0.2417 - acc: 0.944 - ETA: 0s - loss: 0.2359 - acc: 0.946 - ETA: 0s - loss: 0.2415 - acc: 0.944 - ETA: 0s - loss: 0.2457 - acc: 0.941 - ETA: 0s - loss: 0.

1437/1437 [==============================] - ETA: 0s - loss: 0.1927 - acc: 0.940 - ETA: 0s - loss: 0.2102 - acc: 0.944 - ETA: 0s - loss: 0.2039 - acc: 0.950 - ETA: 0s - loss: 0.1851 - acc: 0.951 - ETA: 0s - loss: 0.1788 - acc: 0.949 - ETA: 0s - loss: 0.1713 - acc: 0.952 - ETA: 0s - loss: 0.1664 - acc: 0.955 - ETA: 0s - loss: 0.1638 - acc: 0.956 - ETA: 0s - loss: 0.1584 - acc: 0.960 - 0s 347us/step - loss: 0.1609 - acc: 0.9589
Epoch 47/200
1437/1437 [==============================] - ETA: 0s - loss: 0.1161 - acc: 0.980 - ETA: 0s - loss: 0.1533 - acc: 0.970 - ETA: 0s - loss: 0.1598 - acc: 0.965 - ETA: 0s - loss: 0.1524 - acc: 0.960 - ETA: 0s - loss: 0.1486 - acc: 0.962 - ETA: 0s - loss: 0.1453 - acc: 0.964 - ETA: 0s - loss: 0.1458 - acc: 0.963 - ETA: 0s - loss: 0.1486 - acc: 0.966 - 0s 331us/step - loss: 0.1494 - acc: 0.9645
Epoch 48/200
1437/1437 [==============================] - ETA: 0s - loss: 0.0747 - acc: 0.980 - ETA: 0s - loss: 0.1514 - acc: 0.960 - ETA: 0s - loss: 0.1270 - acc: 0

...

1437/1437 [==============================] - ETA: 0s - loss: 0.0785 - acc: 1.000 - ETA: 0s - loss: 0.0308 - acc: 1.000 - ETA: 0s - loss: 0.0360 - acc: 0.991 - ETA: 0s - loss: 0.0353 - acc: 0.990 - ETA: 0s - loss: 0.0362 - acc: 0.991 - ETA: 0s - loss: 0.0358 - acc: 0.991 - ETA: 0s - loss: 0.0361 - acc: 0.991 - ETA: 0s - loss: 0.0345 - acc: 0.990 - ETA: 0s - loss: 0.0346 - acc: 0.990 - ETA: 0s - loss: 0.0378 - acc: 0.989 - ETA: 0s - loss: 0.0422 - acc: 0.987 - ETA: 0s - loss: 0.0438 - acc: 0.985 - ETA: 0s - loss: 0.0442 - acc: 0.985 - 1s 550us/step - loss: 0.0458 - acc: 0.9840
Epoch 130/200
1437/1437 [==============================] - ETA: 0s - loss: 0.0503 - acc: 1.000 - ETA: 0s - loss: 0.0423 - acc: 0.992 - ETA: 0s - loss: 0.0402 - acc: 0.991 - ETA: 0s - loss: 0.0440 - acc: 0.991 - ETA: 0s - loss: 0.0485 - acc: 0.989 - ETA: 0s - loss: 0.0512 - acc: 0.987 - ETA: 0s - loss: 0.0561 - acc: 0.983 - ETA: 0s - loss: 0.0584 - acc: 0.984 - 0s 339us/step - loss: 0.0565 - acc: 0.9847
Epoch 131/20

1437/1437 [==============================] - ETA: 0s - loss: 0.0549 - acc: 0.980 - ETA: 0s - loss: 0.0664 - acc: 0.966 - ETA: 0s - loss: 0.0506 - acc: 0.975 - ETA: 0s - loss: 0.0480 - acc: 0.976 - ETA: 0s - loss: 0.0473 - acc: 0.977 - ETA: 0s - loss: 0.0471 - acc: 0.980 - ETA: 0s - loss: 0.0450 - acc: 0.982 - ETA: 0s - loss: 0.0402 - acc: 0.985 - ETA: 0s - loss: 0.0426 - acc: 0.984 - ETA: 0s - loss: 0.0459 - acc: 0.981 - ETA: 0s - loss: 0.0521 - acc: 0.980 - ETA: 0s - loss: 0.0490 - acc: 0.981 - ETA: 0s - loss: 0.0497 - acc: 0.981 - ETA: 0s - loss: 0.0503 - acc: 0.980 - 1s 614us/step - loss: 0.0486 - acc: 0.9812
Epoch 143/200
1437/1437 [==============================] - ETA: 0s - loss: 0.1115 - acc: 0.960 - ETA: 0s - loss: 0.0487 - acc: 0.980 - ETA: 0s - loss: 0.0482 - acc: 0.982 - ETA: 0s - loss: 0.0459 - acc: 0.982 - ETA: 0s - loss: 0.0410 - acc: 0.984 - ETA: 0s - loss: 0.0415 - acc: 0.986 - ETA: 0s - loss: 0.0451 - acc: 0.984 - ETA: 0s - loss: 0.0436 - acc: 0.984 - ETA: 0s - loss: 0

1437/1437 [==============================] - ETA: 0s - loss: 0.0408 - acc: 0.980 - ETA: 0s - loss: 0.0395 - acc: 0.990 - ETA: 0s - loss: 0.0416 - acc: 0.986 - ETA: 0s - loss: 0.0518 - acc: 0.982 - ETA: 0s - loss: 0.0564 - acc: 0.981 - ETA: 0s - loss: 0.0516 - acc: 0.983 - ETA: 0s - loss: 0.0458 - acc: 0.985 - ETA: 0s - loss: 0.0465 - acc: 0.984 - ETA: 0s - loss: 0.0458 - acc: 0.985 - ETA: 0s - loss: 0.0445 - acc: 0.986 - ETA: 0s - loss: 0.0445 - acc: 0.984 - ETA: 0s - loss: 0.0472 - acc: 0.983 - 1s 489us/step - loss: 0.0461 - acc: 0.9840
Epoch 158/200
1437/1437 [==============================] - ETA: 0s - loss: 0.0233 - acc: 0.980 - ETA: 0s - loss: 0.0432 - acc: 0.980 - ETA: 0s - loss: 0.0372 - acc: 0.984 - ETA: 0s - loss: 0.0352 - acc: 0.985 - ETA: 0s - loss: 0.0396 - acc: 0.982 - ETA: 0s - loss: 0.0402 - acc: 0.984 - ETA: 0s - loss: 0.0372 - acc: 0.986 - ETA: 0s - loss: 0.0411 - acc: 0.984 - ETA: 0s - loss: 0.0471 - acc: 0.980 - 1s 364us/step - loss: 0.0446 - acc: 0.9819
Epoch 159/20

1437/1437 [==============================] - ETA: 0s - loss: 0.0353 - acc: 1.000 - ETA: 0s - loss: 0.0357 - acc: 0.990 - ETA: 0s - loss: 0.0518 - acc: 0.982 - ETA: 0s - loss: 0.0539 - acc: 0.982 - ETA: 0s - loss: 0.0471 - acc: 0.985 - ETA: 0s - loss: 0.0436 - acc: 0.986 - ETA: 0s - loss: 0.0409 - acc: 0.987 - ETA: 0s - loss: 0.0402 - acc: 0.988 - ETA: 0s - loss: 0.0392 - acc: 0.988 - ETA: 0s - loss: 0.0405 - acc: 0.987 - ETA: 0s - loss: 0.0399 - acc: 0.988 - 1s 478us/step - loss: 0.0422 - acc: 0.9868
Epoch 174/200
1437/1437 [==============================] - ETA: 0s - loss: 0.0465 - acc: 0.980 - ETA: 0s - loss: 0.0383 - acc: 0.980 - ETA: 0s - loss: 0.0387 - acc: 0.980 - ETA: 0s - loss: 0.0404 - acc: 0.980 - ETA: 0s - loss: 0.0345 - acc: 0.984 - ETA: 0s - loss: 0.0367 - acc: 0.983 - ETA: 0s - loss: 0.0385 - acc: 0.982 - ETA: 0s - loss: 0.0442 - acc: 0.980 - ETA: 0s - loss: 0.0422 - acc: 0.980 - 1s 353us/step - loss: 0.0419 - acc: 0.9812
Epoch 175/200
1437/1437 [=========================

1437/1437 [==============================] - ETA: 0s - loss: 0.0086 - acc: 1.000 - ETA: 0s - loss: 0.0200 - acc: 0.995 - ETA: 0s - loss: 0.0353 - acc: 0.986 - ETA: 0s - loss: 0.0338 - acc: 0.986 - ETA: 0s - loss: 0.0313 - acc: 0.989 - ETA: 0s - loss: 0.0356 - acc: 0.989 - ETA: 0s - loss: 0.0394 - acc: 0.988 - ETA: 0s - loss: 0.0376 - acc: 0.989 - ETA: 0s - loss: 0.0366 - acc: 0.990 - ETA: 0s - loss: 0.0379 - acc: 0.989 - ETA: 0s - loss: 0.0418 - acc: 0.987 - ETA: 0s - loss: 0.0437 - acc: 0.985 - 1s 509us/step - loss: 0.0480 - acc: 0.9833
Epoch 192/200
1437/1437 [==============================] - ETA: 0s - loss: 0.0148 - acc: 1.000 - ETA: 0s - loss: 0.0242 - acc: 0.990 - ETA: 0s - loss: 0.0234 - acc: 0.986 - ETA: 0s - loss: 0.0270 - acc: 0.987 - ETA: 0s - loss: 0.0336 - acc: 0.982 - ETA: 0s - loss: 0.0326 - acc: 0.983 - ETA: 0s - loss: 0.0336 - acc: 0.985 - ETA: 0s - loss: 0.0330 - acc: 0.985 - ETA: 0s - loss: 0.0390 - acc: 0.983 - ETA: 0s - loss: 0.0411 - acc: 0.983 - 1s 452us/step - l

In [33]:
# Save model to be used later
agent.persist(model_path)

In [34]:
# Load saved agent, in case no training is needed.
agent = Agent.load('./models/dialogue', interpreter=interpreter, action_endpoint=action_endpoint)

In [5]:
# Generate tree of interaction of utterances in possible dialogs
treeFile = "images/dialogtree.html"
agent.visualize("data/stories.md",output_file=treeFile, max_history=2)
IFrame(src=treeFile, width=1000, height=600)

Processed Story Blocks: 100%|███████████████████████████████████████████| 30/30 [00:00<00:00, 144.90it/s, # trackers=1]


In [18]:
interpreter.parse(u"Pass the remote") # Parsed phrase

{'intent': {'name': 'reach', 'confidence': 0.503009711233227},
 'entities': [{'start': 9,
   'end': 15,
   'value': 'remote',
   'entity': 'object',
   'confidence': 0.9015564182515755,
   'extractor': 'ner_crf'}],
 'intent_ranking': [{'name': 'reach', 'confidence': 0.503009711233227},
  {'name': 'move', 'confidence': 0.21193673058601706},
  {'name': 'call', 'confidence': 0.11101750540756687},
  {'name': 'status', 'confidence': 0.040966139842331675},
  {'name': 'attention', 'confidence': 0.02785851299136267},
  {'name': 'thanks', 'confidence': 0.027307996844929724},
  {'name': 'deny', 'confidence': 0.018137969234439215},
  {'name': 'joke', 'confidence': 0.017821324100716216},
  {'name': 'goodbye', 'confidence': 0.016981845815809575},
  {'name': 'greet', 'confidence': 0.015907384989873076}],
 'text': 'Pass the remote'}

In [6]:
# Interpret a sentence
def parsetxt(sent, deptree = False):
    pp = interpreter.parse(sent) # Parsed phrase
    # Print entities present in the sentence
    print(pd.DataFrame(pp['entities']))
    df = pd.DataFrame(pp['intent_ranking']).style.apply(lambda x: ['background: lightgreen' if x.name == 0 else '' for i in x],axis=1)
    if(deptree):
        doc = nlp(sent)
        displacy.render(doc, style='dep', jupyter=True, options={'distance':90})
    return df

In [7]:
# Interpret a sentence
sent1 = u"Nurse, can you pass me the remote and my book, please"
parsetxt(sent1)

   confidence  end  entity extractor  start   value
0    0.754241   33  object   ner_crf     27  remote
1    0.810362   45  object   ner_crf     41    book


,confidence,name
0,0.554092,reach
1,0.071187,move
2,0.070123,status
3,0.0600482,attention
4,0.0598739,thanks
5,0.0515956,joke
6,0.0436073,call
7,0.0287047,goodbye
8,0.0243287,deny
9,0.0209694,greet


In [21]:
# Render Dependency Tree
doc = nlp(sent1)
displacy.render(doc, style='dep', jupyter=True, options={'distance':90})

In [22]:
# Interprete a sentence
sent2 = "Nurse, help me to get to the bathroom please"
parsetxt(sent2)

   confidence  end entity extractor  start     value
0    0.900694   37  place   ner_crf     29  bathroom


,confidence,name
0,0.449045,move
1,0.185323,joke
2,0.108928,reach
3,0.0658736,call
4,0.0525246,status
5,0.0343769,thanks
6,0.0331593,attention
7,0.0237619,goodbye
8,0.020277,greet
9,0.0177115,deny


In [23]:
# Render Dependency Tree
doc = nlp(sent2)
displacy.render(doc, style='dep', jupyter=True, options={'distance':110})

In [8]:
parsetxt("Call my daughter and call the doctor")

   confidence  end  entity extractor  start     value
0    0.541231   16  callto   ner_crf      8  daughter
1    0.863743   36  callto   ner_crf     30    doctor


,confidence,name
0,0.450662,call
1,0.122582,reach
2,0.102671,move
3,0.0721962,thanks
4,0.054462,status
5,0.0476292,joke
6,0.03903,attention
7,0.0346482,deny
8,0.0327137,greet
9,0.0290438,goodbye


In [25]:
#Script to get the Text-To-Speech output.
audioFolder = "./audios/"
effect = ["",
          "&FX_TYPE=P&FX_LEVEL=2", #Pitch
          "&FX_TYPE=D&FX_LEVEL=-1" #Duration
         ]
rooturl = ["https://cache-a.oddcast.com/tts/gen.php?EID=3&LID=1&VID=3&TXT=",
           "&IS_UTF8=1&ACC=3314795&API=2292376&CB=vw_mc.vwCallback&HTTP_ERR=1&vwApiVersion=2"]
characters ={"," : "%2C"," " : "%20","?" : "%3F","'" : "%27"}
def play(phrase, ef = 0):
    for ch in characters:
        phrase = phrase.replace(ch,characters[ch]) #Replace character to match type in URL
    url = rooturl[0] + phrase + effect[ef]+ rooturl[1] #Build the URL
    file = audioFolder + str(uuid.uuid4()) +'.mp3'
    urllib.request.urlretrieve(url, file)
    playsound(file, True)
    os.remove(file)

In [26]:
play("My name is nurse. How may I help you?",2)

In [17]:
#Configure microphone calibration
# Perform a quick test
r = sr.Recognizer()
with sr.Microphone() as source:
    #print("Please wait. Calibrating microphone...")
    # listen for 1 second and create the ambient noise energy level
    r.adjust_for_ambient_noise(source, duration=1)
    print("Say something!")
    audio = r.listen(source,phrase_time_limit=5)
    response = r.recognize_google(audio)
    print("I think you said: " + response)

Say something!


UnknownValueError: 

In [27]:
def printOut(msg, speech_out = False):
    print("👩 "+" >> " + msg +"\n")
    if(speech_out):
        play(msg)

In [28]:
def init_nurse_bot(speech_in=False,speech_out=False):
    print("[INFO] The nurse is ready to listen. Please start a dialog... (Type 'stop' to quit)\n")
    errormsg = "I'm sorry, I didn't get it. Could you repeat please?"
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source, duration=1)
        while True:
            # >>>>>>>>>>> Input sentence
            if(speech_in):
                flag = False
                cont = 0
                while(not flag):
                    try:
                        print("Listening... >> ", end='')
                        audio = r.listen(source,phrase_time_limit=3)
                        a = r.recognize_google(audio)
                        print(a)
                        flag = True
                    except:
                        cont += 1
                        printOut(errormsg, False)
                        if(cont == 3):
                            print("\n[TERMINATED]", end='')
                            return
            else:
                print(">> ", end='')
                a = input()
            if a == 'stop':
                print("\n[TERMINATED]", end='')
                return
            responses = agent.handle_message(a)
            if(len(responses) == 0):
                print("Action returned 0 elements. Maybe the action listener is not active. Activate listener:\n python -m rasa_core_sdk.endpoint --actions actions")
            for response in responses:
                printOut(response["text"], speech_out)
                if(response["text"] == "Have a nice day!"):
                    print("\n[TERMINATED]", end='')
                    return

In [36]:
init_nurse_bot(False,False)

[INFO] The nurse is ready to listen. Please start a dialog... (Type 'stop' to quit)

>> Hello
👩  >> Hi human. You can call me Nurse. How may I help you?

>> I feel fine
👩  >> Anytime!

👩  >> Is there something else I can do for you?

>> i am feeling bad
Action returned 0 elements. Maybe the action listener is not active. Activate listener:
 python -m rasa_core_sdk.endpoint --actions actions
>> I feel pain
👩  >> That's not good. Let me get some medicine, and I will call a human to verify your condition.

>> stop

[TERMINATED]

In [29]:
#!python -m rasa_core.run -d models/dialogue -u models/nursebot/interpreter --debug --endpoints endpoints.yml